# Import

In [4]:
!pip install wikipedia

In [18]:
!pip install numexpr

In [5]:
#!pip install langchain-community

In [6]:
from langchain.agents import initialize_agent, AgentType, load_tools, tool
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from datetime import datetime

import getpass
import os

"""
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
"""

llm = ChatOpenAI()

# Tool Definition

In [14]:
@tool
def return_current_time(a) -> datetime :
    """"
    Use this function to return the current time ( in the hour, minute, second format)
    input param : None
    return : current time in the format '%H:%M:%S'
    """
    # storing the current time in the variable
    c = datetime.now()
    
    # Displays Time
    current_time = c.strftime('%H:%M:%S')
    return current_time

@tool
def triple(num: float) -> float:
    """
    :param num: a number to triple
    :return: the number tripled ->  multiplied by 3
    """
    return 3 * float(num)


@tool
def return_error(scenario: int):
    """
    something about the funciton
    """
    if scenario ==1:
        return "getting 400 error"
    else:
        return "getting 500 error"

# Assigining Tools to the Agent

In [11]:
predefined_tools = load_tools(['wikipedia'], llm=llm)

tools= [return_current_time,triple ] + predefined_tools 
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    return_intermediate_steps=True,
)

C:\Users\saura\AppData\Local\Temp\ipykernel_16924\2815301781.py:4: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [ ]:
# ImportError: LLMMathChain requires the numexpr package. Please install it with `pip install numexpr`.


## With Wiki, Return Current time & triple

In [12]:
agent.invoke( {"what is the current time? Tell me the hour and multiply by 3 "})



> Entering new AgentExecutor chain...
I should first get the current time and then multiply the hour by 3.
Action: return_current_time
Action Input: None
Observation: 16:54:37
Thought:Now I need to extract the hour from the current time and triple it.
Action: triple
Action Input: 16
Observation: 48.0
Thought:I now know the final answer
Final Answer: 48.0

> Finished chain.


{'input': {'what is the current time? Tell me the hour and multiply by 3 '},
 'output': '48.0',
 'intermediate_steps': [(AgentAction(tool='return_current_time', tool_input='None', log='I should first get the current time and then multiply the hour by 3.\nAction: return_current_time\nAction Input: None'),
   '16:54:37'),
  (AgentAction(tool='triple', tool_input='16', log='Now I need to extract the hour from the current time and triple it.\nAction: triple\nAction Input: 16'),
   48.0)]}

In [13]:
agent.invoke( {"what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 "})



> Entering new AgentExecutor chain...
I need to find out the name of Aizen's Shikai and calculate the square of 1+2+3*2.
Action: wikipedia
Action Input: "Aizen's Shikai"
Observation: Page: Sōsuke Aizen
Summary: Sōsuke Aizen (藍染 惣右介, Aizen Sōsuke) is a fictional character in the Japanese manga series Bleach created by Tite Kubo. He is the main antagonist of the first part of the story of Bleach. Serving as captain of the Fifth Division of Soul Reapers when he is introduced, Aizen later betrays the Soul Society in search of immense power to overthrow and defeat the Soul King, before Ichigo Kurosaki and Kisuke Urahara defeat him. His former lieutenant is Momo Hinamori.

Page: Bleach season 14
Summary: The fourteenth season of the Bleach anime series is based on Tite Kubo's Bleach manga series. It is known as the Arrancar: Downfall arc (破面・滅亡篇, Arankaru Metsubō Hen), is directed by Noriyuki Abe, and produced by TV Tokyo, Dentsu and Studio Pierrot. The season concludes the fight between t

ValidationError: 1 validation error for triple
num
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='1+2+3*2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

### OBSERVATION - without LLM MATH, triple has failed as there needs to be some calculation to be done before passing it to "triple"

### Testing the Return Error problem, without LLM MATH ; with return_error

In [15]:
predefined_tools = load_tools(['wikipedia'], llm=llm)

tools= [return_current_time,triple, return_error ] + predefined_tools 
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    return_intermediate_steps=True,
)

In [16]:
agent.invoke( {"I have just found out that we have a scenario 1 at our hand. Tell me the corresponding error code and triple it"})



> Entering new AgentExecutor chain...
I need to find the error code for scenario 1 and triple it.
Action: return_error
Action Input: 1
Observation: getting 500 error
Thought:I need to triple the error code 500.
Action: triple
Action Input: 500
Observation: 1500.0
Thought:I now know the final answer
Final Answer: 1500.0

> Finished chain.


{'input': {'I have just found out that we have a scenario 1 at our hand. Tell me the corresponding error code and triple it'},
 'output': '1500.0',
 'intermediate_steps': [(AgentAction(tool='return_error', tool_input='1', log='I need to find the error code for scenario 1 and triple it.\nAction: return_error\nAction Input: 1'),
   'getting 500 error'),
  (AgentAction(tool='triple', tool_input='500', log='I need to triple the error code 500.\nAction: triple\nAction Input: 500'),
   1500.0)]}

# TESTING The Aizen question with LLM MATH

In [19]:
# "llm-math"
predefined_tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

tools= [return_current_time,triple, return_error ] + predefined_tools 
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    return_intermediate_steps=True,
)

In [20]:
agent.invoke( {"what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 "})



> Entering new AgentExecutor chain...
I need to find out the name of Aizen's Shikai and calculate the square of 1+2+3*2.
Action: wikipedia
Action Input: "Aizen's Shikai"
Observation: Page: Sōsuke Aizen
Summary: Sōsuke Aizen (藍染 惣右介, Aizen Sōsuke) is a fictional character in the Japanese manga series Bleach created by Tite Kubo. He is the main antagonist of the first part of the story of Bleach. Serving as captain of the Fifth Division of Soul Reapers when he is introduced, Aizen later betrays the Soul Society in search of immense power to overthrow and defeat the Soul King, before Ichigo Kurosaki and Kisuke Urahara defeat him. His former lieutenant is Momo Hinamori.

Page: Bleach season 14
Summary: The fourteenth season of the Bleach anime series is based on Tite Kubo's Bleach manga series. It is known as the Arrancar: Downfall arc (破面・滅亡篇, Arankaru Metsubō Hen), is directed by Noriyuki Abe, and produced by TV Tokyo, Dentsu and Studio Pierrot. The season concludes the fight between t

{'input': {"what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 "},
 'output': "The name of Aizen's Shikai is not explicitly mentioned in the provided information, but the square of 1+2+3*2 is 9.",
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input="Aizen's Shikai", log='I need to find out the name of Aizen\'s Shikai and calculate the square of 1+2+3*2.\nAction: wikipedia\nAction Input: "Aizen\'s Shikai"'),
   'Page: Sōsuke Aizen\nSummary: Sōsuke Aizen (藍染 惣右介, Aizen Sōsuke) is a fictional character in the Japanese manga series Bleach created by Tite Kubo. He is the main antagonist of the first part of the story of Bleach. Serving as captain of the Fifth Division of Soul Reapers when he is introduced, Aizen later betrays the Soul Society in search of immense power to overthrow and defeat the Soul King, before Ichigo Kurosaki and Kisuke Urahara defeat him. His former lieutenant is Momo Hinamori.\n\nPage: Bleach season 14\nSummary: The four

### Observation -- It calculated the expression but didn't square it

In [21]:
# re running it 
agent.invoke( {"what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 "})



> Entering new AgentExecutor chain...
We need to find out the name of Aizen's Shikai and calculate the square of the expression 1+2+3*2.
Action: wikipedia
Action Input: "Aizen's Shikai"
Observation: Page: Sōsuke Aizen
Summary: Sōsuke Aizen (藍染 惣右介, Aizen Sōsuke) is a fictional character in the Japanese manga series Bleach created by Tite Kubo. He is the main antagonist of the first part of the story of Bleach. Serving as captain of the Fifth Division of Soul Reapers when he is introduced, Aizen later betrays the Soul Society in search of immense power to overthrow and defeat the Soul King, before Ichigo Kurosaki and Kisuke Urahara defeat him. His former lieutenant is Momo Hinamori.

Page: Bleach season 14
Summary: The fourteenth season of the Bleach anime series is based on Tite Kubo's Bleach manga series. It is known as the Arrancar: Downfall arc (破面・滅亡篇, Arankaru Metsubō Hen), is directed by Noriyuki Abe, and produced by TV Tokyo, Dentsu and Studio Pierrot. The season concludes the

{'input': {"what is the name of Aizen's Shikai ? Tell me the name and also the square of 1+2+3*2 "},
 'output': "The name of Aizen's Shikai is Kyōka Suigetsu and the square of 1+2+3*2 is 9.",
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input="Aizen's Shikai", log='We need to find out the name of Aizen\'s Shikai and calculate the square of the expression 1+2+3*2.\nAction: wikipedia\nAction Input: "Aizen\'s Shikai"'),
   'Page: Sōsuke Aizen\nSummary: Sōsuke Aizen (藍染 惣右介, Aizen Sōsuke) is a fictional character in the Japanese manga series Bleach created by Tite Kubo. He is the main antagonist of the first part of the story of Bleach. Serving as captain of the Fifth Division of Soul Reapers when he is introduced, Aizen later betrays the Soul Society in search of immense power to overthrow and defeat the Soul King, before Ichigo Kurosaki and Kisuke Urahara defeat him. His former lieutenant is Momo Hinamori.\n\nPage: Bleach season 14\nSummary: The fourteenth season of the Bl

### This time able to get Aizen's shikai name , but still didn't square the expression